In [3]:
import pandas as pd

# Load the datasets
df1 = pd.read_csv('thursday.csv')
df2 = pd.read_csv('tuesday.csv')
df3 = pd.read_csv('wednesday.csv')

# Concatenate the DataFrames
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_csv.csv', index=False)
pd.set_option('display.max_columns', None)

# Now when you use .head(), it will display all columns
merged_df.head()


,id,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd RST Flags,Bwd RST Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWR Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Fwd Bytes/Bulk Avg,Fwd Packet/Bulk Avg,Fwd Bulk Rate Avg,Bwd Bytes/Bulk Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,ICMP Code,ICMP Type,Total TCP Flow Time,Label,Attempted Category
0,1,8.6.0.1-8.0.6.4-0-0-0,8.6.0.1,0,8.0.6.4,0,0,2017-07-06 11:59:05.720779,119945515,150,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000,1.250568,8.050035e+05,5.277837e+06,60127797,0,119945515,8.050035e+05,5.277837e+06,60127797,0,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,0,0,1.250568,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0,0,1,0,0,0,0,0,0,0,6.909777e+06,1.165368e+07,20364643,6,2.436882e+07,2.431326e+07,60127797,5702188,-1,-1,0,BENIGN,-1
1,2,192.168.10.50-192.168.10.3-33898-389-6,192.168.10.50,33898,192.168.10.3,389,6,2017-07-06 11:59:00.490140,113095465,48,24,9668,10012,403,0,201.416667,203.548294,923,316,417.166667,231.080951,174.012282,0.636630,1.592894e+06,4.597265e+06,16482821,3,113095465,2.406286e+06,5.491986e+06,16482821,3,113095083,4.917178e+06,7.084368e+06,16483210,3,24,24,0,0,0,0,1536,768,0.424420,0.212210,0,923,273.333333,234.977733,55214.535211,0,0,0,48,72,0,0,0,0.500000,273.333333,201.416667,417.166667,0,0,0,0,0,0,0,134,0,139,571,2079,23,32,2.039855e+05,5.758373e+05,1629110,379,1.382116e+07,4.277541e+06,16482821,6737603,-1,-1,113095465,BENIGN,-1
2,3,192.168.10.50-192.168.10.3-33904-389-6,192.168.10.50,33904,192.168.10.3,389,6,2017-07-06 11:59:00.491104,113473706,68,40,11364,12718,403,0,167.117647,171.919413,1139,126,317.950000,208.261294,212.225377,0.951762,1.060502e+06,3.813685e+06,16482853,2,113473706,1.693637e+06,4.719143e+06,16482853,2,113427590,2.908400e+06,5.922355e+06,16483212,3,40,40,0,0,0,0,2176,1280,0.599258,0.352505,0,1139,222.981481,199.175449,39670.859467,0,0,0,80,108,0,0,0,0.588235,222.981481,167.117647,317.950000,0,0,0,0,0,0,0,105,0,117,390,2081,39,32,1.783269e+05,5.034269e+05,1424245,325,1.384682e+07,4.229413e+06,16482853,6945512,-1,-1,113473706,BENIGN,-1
3,4,192.168.10.19-192.168.10.3-32794-389-6,192.168.10.19,32794,192.168.10.3,389,6,2017-07-06 12:00:43.263739,118034439,106,61,17896,27110,1313,0,168.830189,225.320575,3069,0,444.426230,470.042036,381.295496,1.414841,7.110508e+05,4.579007e+06,49024651,0,118034439,1.124138e+06,5.726832e+06,49024651,1,117995477,1.966591e+06,7.492251e+06,49025137,2,58,58,0,0,0,0,3408,1968,0.898043,0.516798,0,3069,269.497006,360.120058,129686.456316,0,4,0,116,165,0,0,0,0.575472,269.497006,168.830189,444.426230,0,0,0,0,0,0,0,107,0,162,29200,2078,58,32,1.675640e+06,1.059676e+06,3343206,636188,2.192323e+07,1.681534e+07,49024651,6604342,-1,-1,118034439,BENIGN,-1
4,5,192.168.10.19-192.168.10.3-44766-3268-6,192.168.10.19,44766,192.168.10.3,3268,6,2017-07-06 12:00:44.602892,116446503,72,47,13380,15294,1313,0,185.833333,247.305080

In [6]:
import pandas as pd
import numpy as np  # Import NumPy
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Load the dataset
data = merged_df

# Dropping non-relevant columns including 'Attempted Category'
irrelevant_columns = ['id', 'Flow ID', 'Src IP', 'Dst IP', 'Timestamp', 'Attempted Category']
data_preprocessed = data.drop(columns=irrelevant_columns)

# Checking for missing values
missing_values = data_preprocessed.isnull().sum()

# Handling missing values: assuming that you fill missing values with zeros
data_preprocessed.fillna(0, inplace=True)

# Encoding categorical variables - encoding the 'Label' column
# All instances of 'benign' or 'normal' are coded as 0, and all other labels are coded as 1.
data_preprocessed['Label'] = data_preprocessed['Label'].apply(lambda x: 0 if x in ['BENIGN'] else 1)

# Replace inf/-inf with NaN
data_preprocessed.replace([np.inf, -np.inf], np.nan, inplace=True)

# Handling possible NaN values after replacing inf/-inf
# You can choose to fill NaNs with a specific value or drop the rows/columns with NaNs
data_preprocessed.fillna(0, inplace=True)  # Example: fill with 0

# Normalizing the data
# We'll use StandardScaler for features that seem to be continuous and have a wide range
# For columns with a known range like protocol, icmp code/type we might use MinMaxScaler or leave them as is.

# Columns that we might scale with MinMaxScaler because they have a known bounded range
bounded_columns = ['Protocol', 'ICMP Code', 'ICMP Type']

# Apply MinMaxScaler to bounded columns
scaler_bounded = MinMaxScaler()
data_preprocessed[bounded_columns] = scaler_bounded.fit_transform(data_preprocessed[bounded_columns])

# Remaining columns to be scaled with StandardScaler
remaining_columns = data_preprocessed.columns.difference(bounded_columns + ['Label'])

# Apply StandardScaler to the remaining columns
scaler_remaining = StandardScaler()
data_preprocessed[remaining_columns] = scaler_remaining.fit_transform(data_preprocessed[remaining_columns])

# Checking the preprocessed data
data_preprocessed.head(), missing_values[missing_values > 0]  # Display missing values if any



(   Src Port  Dst Port  Protocol  Flow Duration  Total Fwd Packet  \
 0 -3.069591 -0.172577  0.000000       3.315991          0.152986   
 1 -0.925908 -0.091652  0.352941       3.103275          0.038571   
 2 -0.925529 -0.091652  0.352941       3.115020          0.061005   
 3 -0.995724 -0.091652  0.352941       3.256646          0.103631   
 4 -0.238624  0.507282  0.352941       3.207336          0.065492   
 
    Total Bwd packets  Total Length of Fwd Packet  Total Length of Bwd Packet  \
 0          -0.013143                   -0.045626                   -0.009045   
 1           0.007052                    0.664349                   -0.005336   
 2           0.020515                    0.788896                   -0.004334   
 3           0.038185                    1.268577                    0.000998   
 4           0.026405                    0.936942                   -0.003379   
 
    Fwd Packet Length Max  Fwd Packet Length Min  Fwd Packet Length Mean  \
 0              -0.4

In [7]:
# Assuming `data_preprocessed` is your preprocessed DataFrame
data_preprocessed.to_csv('preprocessed.csv', index=False)


In [8]:
from sklearn.model_selection import train_test_split

# Split the dataset into a 70% training set and a 30% testing set
train_data, test_data = train_test_split(data_preprocessed, test_size=0.3, random_state=42)

# Saving the training and testing sets to CSV files
train_file_path = 'train.csv'
test_file_path = 'test.csv'
train_data.to_csv(train_file_path, index=False)
test_data.to_csv(test_file_path, index=False)



In [9]:


# Filter out the benign instances for the training data
train_data_benign = train_data[train_data['Label'] == 0]

# Save the benign-only training dataset to a CSV file
benign_train_file_path = 'train_benign.csv'
train_data_benign.to_csv(benign_train_file_path, index=False)

benign_train_file_path


'train_benign.csv'